Dynamic programming works by breaking down complex problems into simpler subproblems. Then, finding optimal solutions to these subproblems. Memorization is a method that saves the outcomes of these processes so that the corresponding answers do not need to be computed when they are later needed. Saving solutions save time on the computation of subproblems that have already been encountered. 

In [ ]:
# 时间复杂度：O(n)
# 空间复杂度：O(n)
class Solution:
    def fib(self, n: int) -> int:
       
        # 排除 Corner Case
        if n == 0:
            return 0
        
        # 创建 dp table 
        dp = [0] * (n + 1)

        # 初始化 dp 数组
        dp[0] = 0
        dp[1] = 1

        # 遍历顺序: 由前向后。因为后面要用到前面的状态
        for i in range(2, n + 1):

            # 确定递归公式/状态转移公式
            dp[i] = dp[i - 1] + dp[i - 2]
        
        # 返回答案
        return dp[n]


# 时间复杂度：O(n)
# 空间复杂度：O(1)
class Solution:
    def fib(self, n: int) -> int:
        if n <= 1:
            return n
        
        prev1, prev2 = 0, 1
        
        for _ in range(2, n + 1):
            curr = prev1 + prev2
            prev1, prev2 = prev2, curr
        
        return prev2

例如：有N件物品和一个最多能背重量为W 的背包。第i件物品的重量是weight[i]，得到的价值是value[i] 。每件物品只能用一次，求解将哪些物品装入背包里物品价值总和最大。

dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - weight[i]] + value[i])

i 来表示物品、j表示背包容量。
即dp[i][j] 表示从下标为[0-i]的物品里任意取，放进容量为j的背包，价值总和最大是多少。

<img src="pic/0-1-Knapsack-660.webp" width="50%">

## 二维数组

In [ ]:
## 二维数组

# 物品重量 [w1, w2, ...], 价值 [v1, v2, ...], 背包容量 W
n = len(weights)
dp = [[0] * (W + 1) for _ in range(n + 1)]

# *** 方式1：外层物品，内层容量（标准写法）
for i in range(n):              # Items 0..n-1
    for j in range(W + 1):      # Capacities 0..W
        if j >= weights[i]:     # Direct access to weights[i]
            dp[i+1][j] = max(dp[i][j], dp[i][j-weights[i]] + values[i])
        else:
            dp[i+1][j] = dp[i][j]
            
# 方式2：外层容量，内层物品（顺序可互换）
for j in range(W + 1):      # Capacity from 0 to W
    for i in range(n):      # Items from 0 to n-1
        if weights[i] <= j:
            dp[i+1][j] = max(dp[i][j], dp[i][j - weights[i]] + values[i])
        else:
            dp[i+1][j] = dp[i][j]
# 结果相同，因为 dp[i][j] 只依赖 dp[i-1][...]，顺序不影响。

## 一维数组
### 01背包

对于背包问题其实状态都是可以压缩的。

在使用二维数组的时候，递推公式：dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - weight[i]] + value[i]);

动态规划中dp[j]是由dp[j-weight[i]]推导出来的，然后取max(dp[j], dp[j - weight[i]] + value[i])

In [ ]:
## 一维数组
### 01背包
# 0/1 Knapsack (Each item can be used at most once)

# *** 正确写法（01背包）
dp = [0] * (W + 1)

for i in range(n):
    for j in range(W, weights[i] - 1, -1):  # 逆序
        dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# 为什么正确？
    # 外层循环物品，内层逆序容量：
    # 每个物品 i 只会被处理一次。
    # 由于容量是 逆序更新，dp[j - weights[i]] 总是来自 上一轮（未包含当前物品） 的值，确保不会重复选取。


# 错误写法（正序）：
for i in range(n):
    for j in range(weights[i], W + 1):  # 正序
        dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# 问题：dp[j - weights[i]] 可能已经被当前物品更新过，导致重复选取。
# Why reverse order?
    # If we go left-to-right, dp[j - w[i]] may have already been updated in the current iteration, leading to multiple picks of the same item.
    # Example (if we go left-to-right):
    # For i=0 (item 1, w=1, v=15):
    # dp[1] = max(dp[1], dp[0] + 15) = 15
    # dp[2] = max(dp[2], dp[1] + 15) = 30 (item 1 picked twice!)
    # This is wrong for 0/1 Knapsack (but correct for Unbounded Knapsack).


# 错误写法（交换顺序）Incorrect Order：
for j in range(W + 1):  # 外层容量 Outer: capacity
    for i in range(n):  # 内层物品 Inner: items
        if weights[i] <= j:
            dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# 问题：dp[j] 会多次更新，导致同一物品被选多次（变成完全背包）。
# What happens?
    # When j=2:
    #     dp[2] = max(dp[2], dp[0] + 20) = 20 (pick item 2)
    # When j=4:
    #     dp[4] = max(dp[4], dp[2] + 20) = 40 (item 2 picked again!)
    # This violates the 0/1 constraint (each item can be used at most once).
# 为什么错误？
    # 问题1：同一物品可能在 同一轮容量更新中被多次选取。
    # 问题2：dp[j - weights[i]] 可能已经被 当前物品更新过，导致重复计算。



# 对于任意物品 i 和容量 j：
# 逆序更新：
# dp[j] 只依赖 dp[j - w[i]]（上一轮的值）。
# 因此，物品 i 不会被重复加入。
# 正序更新：
# dp[j] 可能依赖已经被当前物品更新过的 dp[j - w[i]]。
# 导致物品 i 被多次选取（变成完全背包问题）。

### 完全背包

In [ ]:
### 完全背包
# Unbounded Knapsack (Items can be reused)

# *** 正确写法1（外层物品，内层容量正序）1D DP - Must Forward Iterate
dp = [0] * (W + 1)

for i in range(n):
    for j in range(weights[i], W + 1):  # 正序
        dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# Why forward order?
    # We want to reuse items, so dp[j - w[i]] should include the current item if possible.
    # Example (for i=0, item 1, w=1, v=15):
    # dp[1] = max(0, dp[0] + 15) = 15
    # dp[2] = max(0, dp[1] + 15) = 30 (item 1 used twice)
    # dp[3] = 45, ..., dp[5] = 75 (item 1 used five times).

# 正确写法2（外层容量，内层物品）：
for j in range(W + 1):
    for i in range(n):
        if weights[i] <= j:
            dp[j] = max(dp[j], dp[j - weights[i]] + values[i])
# 结果相同（求最大价值时）。

# Wrong Approach (1D DP - Reverse Order)
dp = [0] * (W + 1)

for i in range(n):
    for j in range(W, w[i] - 1, -1):  # Reverse order (like 0/1 Knapsack)
        dp[j] = max(dp[j], dp[j - w[i]] + v[i])

print(dp[W])  # Output: 65 (Same as 0/1 Knapsack - Wrong for Unbounded!)
# What happens?
    # Reverse order prevents reusing items (like 0/1 Knapsack).
    # We get the 0/1 Knapsack solution (65) instead of the unbounded solution (75).

In [ ]:
322. Coin Change

## (1) 组合数（顺序不重要）

二维DP数组递推公式： dp[i][j] = dp[i - 1][j] + dp[i - 1][j - nums[i]];
去掉维度i 之后，递推公式：dp[j] = dp[j] + dp[j - nums[i]] ，即：dp[j] += dp[j - nums[i]]

In [ ]:
# 3. 组合数 vs 排列数（完全背包变种）
# (1) 组合数（顺序不重要）
# 问题：w = [1, 2, 3]，W = 4，求有多少种组合方式？
dp = [0] * (W + 1) # dp[j] 表示凑出重量 j 的组合数
dp[0] = 1  # 初始化：凑出重量 0 的组合数为 1（即不选任何物品）
# dp = [1, 0, 0, 0, 0]（W=4，所以数组长度为 5）。


# 外层物品，内层容量（组合数）
for i in range(n):
    # 对当前物品 weights[i]，从 j = weights[i] 开始更新到 W。
    # 正序更新：确保可以重复使用当前物品（但组合数问题中顺序不重要）。
    for j in range(weights[i], W + 1):
        dp[j] += dp[j - weights[i]]
# 逻辑：如果当前物品 weights[i] 可以放入容量 j，则组合数增加 dp[j - weights[i]]。
# 物理意义：
    # dp[j - weights[i]] 是 不选当前物品时 凑出 j - weights[i] 的组合数。
    # 现在选了当前物品，组合数自然累加。
    # dp[j] += ...：如果选了当前物品 weights[i]，那么凑出 j 的组合数 增加 dp[j - weights[i]] 种方式。

# 为什么外层循环是物品，内层循环是容量？
    # 外层循环物品：确保 组合的唯一性（顺序不重要）。
    # 例如，[1, 2] 和 [2, 1] 不会被重复计算，因为 2 是在 1 之后处理的。
    # 内层循环容量（正序）：允许 重复使用当前物品（完全背包特性）。

# 为什么能保证顺序不重要？
    # 外层按物品顺序处理，固定了物品的选择顺序。
    # 例如，在处理完物品 1 后，再处理物品 2，因此 [1,2] 会被计算，但 [2,1] 不会重复出现。

print(dp[W])  # 输出：4（[1,1,1,1], [1,1,2], [1,3], [2,2]）
# 说明：[1,1,2] 和 [2,1,1] 算作同一种组合。

# Step 1: 处理物品 1（重量 1）
# for j in range(1, 5):  # j = 1, 2, 3, 4
#     dp[j] += dp[j - 1]
# j=1：dp[1] += dp[0] → dp = [1, 1, 0, 0, 0]（组合 [1]）
# j=2：dp[2] += dp[1] → dp = [1, 1, 1, 0, 0]（组合 [1,1]）
# j=3：dp[3] += dp[2] → dp = [1, 1, 1, 1, 0]（组合 [1,1,1]）
# j=4：dp[4] += dp[3] → dp = [1, 1, 1, 1, 1]（组合 [1,1,1,1]）  ***

# Step 2: 处理物品 2（重量 2）
# for j in range(2, 5):  # j = 2, 3, 4
#     dp[j] += dp[j - 2]
# j=2：dp[2] += dp[0] → dp = [1, 1, 2, 1, 1]（新增组合 [2]）
# j=3：dp[3] += dp[1] → dp = [1, 1, 2, 2, 1]（新增组合 [1,2]）
# j=4：dp[4] += dp[2] → dp = [1, 1, 2, 2, 3]（新增组合 [2,2] 和 [1,1,2]） ***

# Step 3: 处理物品 3（重量 3）
# for j in range(3, 5):  # j = 3, 4
#     dp[j] += dp[j - 3]
# j=3：dp[3] += dp[0] → dp = [1, 1, 2, 3, 3]（新增组合 [3]）
# j=4：dp[4] += dp[1] → dp = [1, 1, 2, 3, 4]（新增组合 [1,3]） ***
# 最终结果
# dp = [1, 1, 2, 3, 4]  # dp[4] = 4
# 组合方式：
# [1,1,1,1]
# [1,1,2]
# [2,2]
# [1,3]

# 具体例子
# 假设 weights = [1, 2, 3]，W = 4，我们来看如何计算 dp[4]：
# (1) 处理物品 1（重量 1）
# dp[4] += dp[4 - 1]（即 dp[4] += dp[3]）
# dp[3] 表示凑出 3 的组合数，现在加上 1 就能凑出 4。
# 例如：如果 dp[3] 包含 [1,1,1] 和 [3]，那么加上 1 后变成 [1,1,1,1] 和 [3,1]。

# (2) 处理物品 2（重量 2）
# dp[4] += dp[4 - 2]（即 dp[4] += dp[2]）
# dp[2] 表示凑出 2 的组合数（如 [1,1] 和 [2]），加上 2 后变成 [1,1,2] 和 [2,2]。

# (3) 处理物品 3（重量 3）
# dp[4] += dp[4 - 3]（即 dp[4] += dp[1]）
# dp[1] 表示凑出 1 的组合数（如 [1]），加上 3 后变成 [1,3]。

In [ ]:
518. Coin Change II

## (2) 排列数（顺序重要）

In [ ]:
# (2) 排列数（顺序重要）
# 问题：w = [1, 2, 3]，W = 4，求有多少种排列方式？
dp = [0] * (W + 1)
dp[0] = 1  # 初始化

# 外层容量，内层物品（排列数）
for j in range(W + 1):
    for i in range(n):
        if w[i] <= j:
            dp[j] += dp[j - w[i]]

print(dp[W])  # 输出：7（[1,1,1,1], [1,1,2], [1,2,1], [2,1,1], [2,2], [1,3], [3,1]）
# 说明：[1,1,2] 和 [2,1,1] 算作不同排列。

# 具体例子分析（w = [1, 2], W = 3）
# 目标：凑出总和为3的所有方式（数字可重复使用）。
# (1) 排列数代码的执行过程
# 初始化：dp = [1, 0, 0, 0]

# 1. 当 j = 1（和为1）
# 遍历物品 [1, 2, 3]：
    # 物品 1（1 ≤ 1）：
    # dp[1] += dp[0] → dp = [1, 1, 0, 0, 0]
    # 新增排列：[1]
    
    # 物品 2（2 > 1）、物品 3（3 > 1）：跳过

# 2. 当 j = 2（和为2）
# 遍历物品 [1, 2, 3]：
    # 物品 1（1 ≤ 2）：
    # dp[2] += dp[1] → dp = [1, 1, 1, 0, 0]
    # 新增排列：[1,1]
    
    # 物品 2（2 ≤ 2）：
    # dp[2] += dp[0] → dp = [1, 1, 2, 0, 0]
    # 新增排列：[2]
    
    # 物品 3（3 > 2）：跳过

# 3. 当 j = 3（和为3）
# 遍历物品 [1, 2, 3]：
    # 物品 1（1 ≤ 3）：
    # dp[3] += dp[2] → dp = [1, 1, 2, 2, 0]
    # 新增排列：[1,1,1] 和 [2,1]
    
    # 物品 2（2 ≤ 3）：
    # dp[3] += dp[1] → dp = [1, 1, 2, 3, 0]
    # 新增排列：[1,2]
    
    # 物品 3（3 ≤ 3）：
    # dp[3] += dp[0] → dp = [1, 1, 2, 4, 0]
    # 新增排列：[3]

# 4. 当 j = 4（和为4）
# 遍历物品 [1, 2, 3]：
    # 物品 1（1 ≤ 4）：
    # dp[4] += dp[3] → dp = [1, 1, 2, 4, 4]
    # 新增排列：[1,1,1,1]、[2,1,1]、[1,2,1]、[3,1] ***
    
    # 物品 2（2 ≤ 4）：
    # dp[4] += dp[2] → dp = [1, 1, 2, 4, 6]
    # 新增排列：[1,1,2]、[2,2] ***
    
    # 物品 3（3 ≤ 4）：
    # dp[4] += dp[1] → dp = [1, 1, 2, 4, 7]
    # 新增排列：[1,3] ***

377. Combination Sum IV

In [ ]:
#24 (Easy) 121. 买卖股票的最佳时机
    # 给定一个数组 prices ，它的第 i 个元素 prices[i] 表示一支给定股票第 i 天的价格。
    # 你只能选择 某一天 买入这只股票，并选择在 未来的某一个不同的日子 卖出该股票。设计一个算法来计算你所能获取的最大利润。
    # 返回你可以从这笔交易中获取的最大利润。如果你不能获取任何利润，返回 0 。
    # 示例 1：
    # 输入：[7,1,5,3,6,4]
    # 输出：5
    # 解释：在第 2 天（股票价格 = 1）的时候买入，在第 5 天（股票价格 = 6）的时候卖出，最大利润 = 6-1 = 5 。注意利润不能是 7-1 = 6, 因为卖出价格需要大于买入价格；同时，你不能在买入前卖出股票。
    # 示例 2：
    # 输入：prices = [7,6,4,3,1]
    # 输出：0
    # 解释：在这种情况下, 没有交易完成, 所以最大利润为 0。
# dp数组的含义:
# dp[i][0] 表示第i天持有股票所得现金。
# dp[i][1] 表示第i天不持有股票所得最多现金
# 注意这里说的是“持有”,“持有”不代表就是当天“买入”!也有可能是昨天就买入了,今天保持持有的状态
# 很多同学把“持有”和“买入”没区分清楚。
# 如果第i天持有股票即dp[i][0]， 那么可以由两个状态推出来
# 第i-1天就持有股票，那么就保持现状，所得现金就是昨天持有股票的所得现金 即：dp[i - 1][0]
# 第i天买入股票，所得现金就是买入今天的股票后所得现金即：-prices[i]
# 那么dp[i][0]应该选所得现金最大的，所以dp[i][0] = max(dp[i - 1][0], -prices[i]);
# 如果第i天不持有股票即dp[i][1]， 也可以由两个状态推出来
# 第i-1天就不持有股票，那么就保持现状，所得现金就是昨天不持有股票的所得现金 即：dp[i - 1][1]
# 第i天卖出股票，所得现金就是按照今天股票价格卖出后所得现金即：prices[i] + dp[i - 1][0]
# 同样dp[i][1]取最大的，dp[i][1] = max(dp[i - 1][1], prices[i] + dp[i - 1][0]);
# 贪心法
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        low = float("inf")
        result = 0
        for i in range(len(prices)):
            low = min(low, prices[i]) #取最左最小价格
            result = max(result, prices[i] - low) #直接取最大区间利润
        return result
# *** 动态规划:版本一
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        length = len(prices)
        if len == 0:
            return 0
        dp = [[0] * 2 for _ in range(length)]
        dp[0][0] = -prices[0]
        dp[0][1] = 0
        for i in range(1, length):
            dp[i][0] = max(dp[i-1][0], -prices[i])
            dp[i][1] = max(dp[i-1][1], prices[i] + dp[i-1][0])
        return dp[-1][1]
# 动态规划:版本二
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        length = len(prices)
        dp = [[0] * 2 for _ in range(2)] #注意这里只开辟了一个2 * 2大小的二维数组
        dp[0][0] = -prices[0]
        dp[0][1] = 0
        for i in range(1, length):
            dp[i % 2][0] = max(dp[(i-1) % 2][0], -prices[i])
            dp[i % 2][1] = max(dp[(i-1) % 2][1], prices[i] + dp[(i-1) % 2][0])
        return dp[(length-1) % 2][1]

In [ ]:
#4 (Medium) 122.买卖股票的最佳时机II
    # 给定一个数组，它的第  i 个元素是一支给定股票第 i 天的价格。
    # 设计一个算法来计算你所能获取的最大利润。你可以尽可能地完成更多的交易（多次买卖一支股票）。
    # 注意：你不能同时参与多笔交易（你必须在再次购买前出售掉之前的股票）。
    # 示例 1:
    # 输入: [7,1,5,3,6,4]
    # 输出: 7
    # 解释: 在第 2 天（股票价格 = 1）的时候买入，在第 3 天（股票价格 = 5）的时候卖出, 这笔交易所能获得利润 = 5-1 = 4。随后，在第 4 天（股票价格 = 3）的时候买入，在第 5 天（股票价格 = 6）的时候卖出, 这笔交易所能获得利润 = 6-3 = 3 。
    # 示例 2:
    # 输入: [1,2,3,4,5]
    # 输出: 4
    # 解释: 在第 1 天（股票价格 = 1）的时候买入，在第 5 天 （股票价格 = 5）的时候卖出, 这笔交易所能获得利润 = 5-1 = 4 。注意你不能在第 1 天和第 2 天接连购买股票，之后再将它们卖出。因为这样属于同时参与了多笔交易，你必须在再次购买前出售掉之前的股票。
    # 示例  3:
    # 输入: [7,6,4,3,1]
    # 输出: 0
    # 解释: 在这种情况下, 没有交易完成, 所以最大利润为 0。
# 第一天当然没有利润，至少要第二天才会有利润，所以利润的序列比股票序列少一天！
# 从图中可以发现，其实我们需要收集每天的正利润就可以，收集正利润的区间，就是股票买卖的区间，而我们只需要关注最终利润，不需要记录区间。
# 那么只收集正利润就是贪心所贪的地方！
# 贪心
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        result = 0
        for i in range(1, len(prices)):
            result += max(prices[i] - prices[i - 1], 0)
        return result
# 动态规划
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        length = len(prices)
        dp = [[0] * 2 for _ in range(length)]
        dp[0][0] = -prices[0]
        dp[0][1] = 0
        for i in range(1, length):
            dp[i][0] = max(dp[i-1][0], dp[i-1][1] - prices[i]) #注意这里是和121. 买卖股票的最佳时机唯一不同的地方
            dp[i][1] = max(dp[i-1][1], dp[i-1][0] + prices[i])
        return dp[-1][1]
# *** 版本二
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        length = len(prices)
        dp = [[0] * 2 for _ in range(2)] #注意这里只开辟了一个2 * 2大小的二维数组
        dp[0][0] = -prices[0]
        dp[0][1] = 0
        for i in range(1, length):
            dp[i % 2][0] = max(dp[(i-1) % 2][0], dp[(i-1) % 2][1] - prices[i])
            dp[i % 2][1] = max(dp[(i-1) % 2][1], dp[(i-1) % 2][0] + prices[i])
        return dp[(length-1) % 2][1]

In [ ]:
#28 ??? (Medium) 309.最佳买卖股票时机含冷冻期
    # 给定一个整数数组，其中第 i 个元素代表了第 i 天的股票价格 。
    # 设计一个算法计算出最大利润。在满足以下约束条件下，你可以尽可能地完成更多的交易（多次买卖一支股票）:
    # 你不能同时参与多笔交易（你必须在再次购买前出售掉之前的股票）。
    # 卖出股票后，你无法在第二天买入股票 (即冷冻期为 1 天)。
    # 示例:
    # 输入: [1,2,3,0,2]
    # 输出: 3
    # 解释: 对应的交易状态为: [买入, 卖出, 冷冻期, 买入, 卖出]
# dp[i][j]，第i天状态为j，所剩的最多现金为dp[i][j]。
# 其实本题很多同学搞的比较懵，是因为出现冷冻期之后，状态其实是比较复杂度，例如今天买入股票、今天卖出股票、今天是冷冻期，都是不能操作股票的。
# 具体可以区分出如下四个状态：
# 状态一：持有股票状态（今天买入股票，或者是之前就买入了股票然后没有操作，一直持有）
# 不持有股票状态，这里就有两种卖出股票状态
# 状态二：保持卖出股票的状态（两天前就卖出了股票，度过一天冷冻期。或者是前一天就是卖出股票状态，一直没操作）
# 状态三：今天卖出股票
# 状态四：今天为冷冻期状态，但冷冻期状态不可持续，只有一天！
# 达到买入股票状态（状态一）即：dp[i][0]，有两个具体操作：
# 操作一：前一天就是持有股票状态（状态一），dp[i][0] = dp[i - 1][0]
# 操作二：今天买入了，有两种情况
# 前一天是冷冻期（状态四），dp[i - 1][3] - prices[i]
# 前一天是保持卖出股票的状态（状态二），dp[i - 1][1] - prices[i]
# 那么dp[i][0] = max(dp[i - 1][0], dp[i - 1][3] - prices[i], dp[i - 1][1] - prices[i]);
# 达到保持卖出股票状态（状态二）即：dp[i][1]，有两个具体操作：
# 操作一：前一天就是状态二
# 操作二：前一天是冷冻期（状态四）
# dp[i][1] = max(dp[i - 1][1], dp[i - 1][3]);
# 达到今天就卖出股票状态（状态三），即：dp[i][2] ，只有一个操作：
# 昨天一定是持有股票状态（状态一），今天卖出
# 即：dp[i][2] = dp[i - 1][0] + prices[i];
# 达到冷冻期状态（状态四），即：dp[i][3]，只有一个操作：
# 昨天卖出了股票（状态三）
# dp[i][3] = dp[i - 1][2];
# 版本一
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n = len(prices)
        if n == 0:
            return 0
        dp = [[0] * 4 for _ in range(n)]  # 创建动态规划数组,4个状态分别表示持有股票、不持有股票且处于冷冻期、不持有股票且不处于冷冻期、不持有股票且当天卖出后处于冷冻期
        dp[0][0] = -prices[0]  # 初始状态:第一天持有股票的最大利润为买入股票的价格
        for i in range(1, n):
            dp[i][0] = max(dp[i-1][0], max(dp[i-1][3], dp[i-1][1]) - prices[i])  # 当前持有股票的最大利润等于前一天持有股票的最大利润或者前一天不持有股票且不处于冷冻期的最大利润减去当前股票的价格
            dp[i][1] = max(dp[i-1][1], dp[i-1][3])  # 当前不持有股票且处于冷冻期的最大利润等于前一天持有股票的最大利润加上当前股票的价格
            dp[i][2] = dp[i-1][0] + prices[i]  # 当前不持有股票且不处于冷冻期的最大利润等于前一天不持有股票的最大利润或者前一天处于冷冻期的最大利润
            dp[i][3] = dp[i-1][2]  # 当前不持有股票且当天卖出后处于冷冻期的最大利润等于前一天不持有股票且不处于冷冻期的最大利润
        return max(dp[n-1][3], dp[n-1][1], dp[n-1][2])  # 返回最后一天不持有股票的最大利润
# 优化
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        n = len(prices)
        if n == 0:
            return 0
        dp = [[0] * 4 for _ in range(2)]  # 创建动态规划数组,4个状态分别表示持有股票、不持有股票且处于冷冻期、不持有股票且不处于冷冻期、不持有股票且当天卖出后处于冷冻期
        dp[0][0] = -prices[0]  # 初始状态:第一天持有股票的最大利润为买入股票的价格
        for i in range(1, n):
            dp[i%2][0] = max(dp[((i-1)%2)%2][0], max(dp[(i-1)%2][3], dp[(i-1)%2][1]) - prices[i])  # 当前持有股票的最大利润等于前一天持有股票的最大利润或者前一天不持有股票且不处于冷冻期的最大利润减去当前股票的价格
            dp[i%2][1] = max(dp[(i-1)%2][1], dp[(i-1)%2][3])  # 当前不持有股票且处于冷冻期的最大利润等于前一天持有股票的最大利润加上当前股票的价格
            dp[i%2][2] = dp[(i-1)%2][0] + prices[i]  # 当前不持有股票且不处于冷冻期的最大利润等于前一天不持有股票的最大利润或者前一天处于冷冻期的最大利润
            dp[i%2][3] = dp[(i-1)%2][2]  # 当前不持有股票且当天卖出后处于冷冻期的最大利润等于前一天不持有股票且不处于冷冻期的最大利润
        return max(dp[(n-1)%2][3], dp[(n-1)%2][1], dp[(n-1)%2][2])  # 返回最后一天不持有股票的最大利润

In [ ]:
#30 (Medium) 300.最长递增子序列 (不连续) Longest Increasing Subsequence
    # 给你一个整数数组 nums ，找到其中最长严格递增子序列的长度。
    # 子序列是由数组派生而来的序列，删除（或不删除）数组中的元素而不改变其余元素的顺序。
    # 例如，[3,6,2,7] 是数组 [0,3,1,6,2,2,7] 的子序列。
    # 示例 1：
    # 输入：nums = [10,9,2,5,3,7,101,18]
    # 输出：4
    # 解释：最长递增子序列是 [2,3,7,101]，因此长度为 4 。
    # 示例 2：
    # 输入：nums = [0,1,0,3,2,3]
    # 输出：4
    # 示例 3：
    # 输入：nums = [7,7,7,7,7,7,7]
    # 输出：1
# dp[i] = max(dp[i], dp[j] + 1)
# 子序列是由数组派生而来的序列，删除（或不删除）数组中的元素而不改变其余元素的顺序
# dp[i]表示i之前包括i的以nums[i]结尾的最长递增子序列的长度
# 为什么一定表示 “以nums[i]结尾的最长递增子序” ，因为我们在 做 递增比较的时候，如果比较 nums[j] 和 nums[i] 的大小，
# 那么两个递增子序列一定分别以nums[j]为结尾 和 nums[i]为结尾， 要不然这个比较就没有意义了，不是尾部元素的比较那么 如何算递增呢。
# 位置i的最长升序子序列等于j从0到i-1各个位置的最长升序子序列 + 1 的最大值。
# 所以：if (nums[i] > nums[j]) dp[i] = max(dp[i], dp[j] + 1)
# 注意这里不是要dp[i] 与 dp[j] + 1进行比较，而是我们要取dp[j] + 1的最大值。
# DP
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        if len(nums) <= 1:
            return len(nums)
        # dp[i]表示i之前包括i的以nums[i]结尾的最长递增子序列的长度
        dp = [1] * len(nums)
        result = 1
        for i in range(1, len(nums)):
            for j in range(0, i):
                if nums[i] > nums[j]:
                    dp[i] = max(dp[i], dp[j] + 1)
            result = max(result, dp[i]) #取长的子序列
        return result

In [ ]:
#31 (Easy) 674.最长连续递增序列 (连续) Longest Continuous Increasing Subsequence
    # 给定一个未经排序的整数数组，找到最长且 连续递增的子序列，并返回该序列的长度。
    # 连续递增的子序列 可以由两个下标 l 和 r（l < r）确定，如果对于每个 l <= i < r，都有 nums[i] < nums[i + 1] ，那么子序列 [nums[l], nums[l + 1], ..., nums[r - 1], nums[r]] 就是连续递增子序列。
    # 示例 1：
    # 输入：nums = [1,3,5,4,7]
    # 输出：3
    # 解释：最长连续递增序列是 [1,3,5], 长度为3。尽管 [1,3,5,7] 也是升序的子序列, 但它不是连续的，因为 5 和 7 在原数组里被 4 隔开。
    # 示例 2：
    # 输入：nums = [2,2,2,2,2]
    # 输出：1
    # 解释：最长连续递增序列是 [2], 长度为1。
# dp[i+1] = dp[i] + 1
# dp[i]：以下标i为结尾的连续递增的子序列长度为dp[i]。
# 注意这里的定义，一定是以下标i为结尾，并不是说一定以下标0为起始位置。
# 如果 nums[i] > nums[i - 1]，那么以 i 为结尾的连续递增的子序列长度 一定等于 以i - 1为结尾的连续递增的子序列长度 + 1 。
# 即：dp[i] = dp[i - 1] + 1;
# 概括来说：不连续递增子序列的跟前0-i 个状态有关，连续递增的子序列只跟前一个状态有关
# DP
class Solution:
    def findLengthOfLCIS(self, nums: List[int]) -> int:
        if len(nums) == 0:
            return 0
        result = 1
        # dp[i]：以下标i为结尾的连续递增的子序列长度为dp[i]。
        dp = [1] * len(nums)
        for i in range(len(nums)-1):
            if nums[i+1] > nums[i]: #连续记录
                dp[i+1] = dp[i] + 1
            result = max(result, dp[i+1])
        return result

In [ ]:
#X32 *** (Medium) 718.最长重复子数组 (连续) Maximum Length of Repeated Subarray
# 两个整数数组
    # 给两个整数数组 A 和 B ，返回两个数组中公共的、长度最长的子数组sub array的长度。
    # 示例：
    # 输入：
    # A: [1,2,3,2,1]
    # B: [3,2,1,4,7]
    # 输出：3
    # 解释：长度最长的公共子数组是 [3, 2, 1] 。
# dp[j] = dp[j - 1] + 1
# dp[i][j] ：以下标i - 1为结尾的A，和以下标j - 1为结尾的B，最长重复子数组长度为dp[i][j]。 
# （特别注意： “以下标i - 1为结尾的A” 标明一定是 以A[i-1]为结尾的字符串 ）
# 此时细心的同学应该发现，那dp[0][0]是什么含义呢？总不能是以下标-1为结尾的A数组吧。
# 其实dp[i][j]的定义也就决定着，我们在遍历dp[i][j]的时候i 和 j都要从1开始。
# 那有同学问了，我就定义dp[i][j]为 以下标i为结尾的A，和以下标j 为结尾的B，最长重复子数组长度。不行么？
# 行倒是行！ 但实现起来就麻烦一点，需要单独处理初始化部分，在本题解下面的拓展内容里，我给出了 第二种 
# dp数组的定义方式所对应的代码和讲解，大家比较一下就了解了。
# 根据dp[i][j]的定义，dp[i][j]的状态只能由dp[i - 1][j - 1]推导出来。
# 即当A[i - 1] 和B[j - 1]相等的时候，dp[i][j] = dp[i - 1][j - 1] + 1;
# 注意题目中说的子数组,其实就是连续子序列。
# 2维DP
class Solution:
    def findLength(self, nums1: List[int], nums2: List[int]) -> int:
        # 创建一个二维数组 dp,用于存储最长公共子数组的长度
        # dp[i][j] ：以下标i - 1为结尾的A，和以下标j - 1为结尾的B，最长重复子数组长度为dp[i][j]
        dp = [[0] * (len(nums2) + 1) for _ in range(len(nums1) + 1)]
        # 记录最长公共子数组的长度
        result = 0

        # 遍历数组 nums1
        for i in range(1, len(nums1) + 1):
            # 遍历数组 nums2
            for j in range(1, len(nums2) + 1):
                # 如果 nums1[i-1] 和 nums2[j-1] 相等
                if nums1[i - 1] == nums2[j - 1]:
                    # 在当前位置上的最长公共子数组长度为前一个位置上的长度加一
                    dp[i][j] = dp[i - 1][j - 1] + 1
                # 更新最长公共子数组的长度
                if dp[i][j] > result:
                    result = dp[i][j]

        # 返回最长公共子数组的长度
        return result
# 1维DP，内层遍历需要从后向前
class Solution:
    def findLength(self, nums1: List[int], nums2: List[int]) -> int:
        dp = [0] * (len(nums2) + 1)
        result = 0

        for i in range(1, len(nums1) + 1):
            # for j in range(1, len(nums2) + 1):
            for j in range(len(nums2), 0, -1):
                if nums1[i - 1] == nums2[j - 1]:
                    dp[j] = dp[j - 1] + 1
                else:
                    dp[j] = 0 #注意这里不相等的时候要有赋0的操作
                result = max(result, dp[j])

        # 返回最长公共子数组的长度
        return result
# *** 1维DP 用prev, curr指针，内层遍历就不用从后向前了
class Solution:
    def findLength(self, nums1: List[int], nums2: List[int]) -> int:
        # 创建一个一维数组 dp,用于存储最长公共子数组的长度
        dp = [0] * (len(nums2) + 1)
        # 记录最长公共子数组的长度
        result = 0

        # 遍历数组 nums1
        for i in range(1, len(nums1) + 1):
            # 用于保存上一个位置的值
            prev = 0
            # 遍历数组 nums2
            for j in range(1, len(nums2) + 1):
                # 保存当前位置的值,因为会在后面被更新
                current = dp[j]
                # 如果 nums1[i-1] 和 nums2[j-1] 相等
                if nums1[i - 1] == nums2[j - 1]:
                    # 在当前位置上的最长公共子数组长度为上一个位置的长度加一
                    dp[j] = prev + 1
                    # 更新最长公共子数组的长度
                    if dp[j] > result:
                        result = dp[j]
                else:
                    # 如果不相等,将当前位置的值置为零
                    dp[j] = 0
                # 更新 prev 变量为当前位置的值,供下一次迭代使用
                prev = current

        # 返回最长公共子数组的长度
        return result

In [ ]:
#37 *** (Hard) 115.不同的子序列 Distinct Subsequences
    # 给定一个字符串 s 和一个字符串 t ，计算在 s 的子序列中 t 出现的个数。
    # 字符串的一个 子序列 是指，通过删除一些（也可以不删除）字符且不干扰剩余字符相对位置所组成的新字符串。
    # （例如，"ACE" 是 "ABCDE" 的一个子序列，而 "AEC" 不是）
    # 题目数据保证答案符合 32 位带符号整数范围。
# dp[i][j] = dp[i-1][j-1] + dp[i-1][j]
# 这道题目如果不是子序列,而是要求连续序列的,那就可以考虑用KMP。
# dp[i][j]：以i-1为结尾的s子序列中出现以j-1为结尾的t的个数为dp[i][j]。
# 这一类问题，基本是要分析两种情况
# s[i - 1] 与 t[j - 1]相等
# s[i - 1] 与 t[j - 1] 不相等
# 当s[i - 1] 与 t[j - 1]相等时，dp[i][j]可以有两部分组成。
# 一部分是用s[i - 1]来匹配，那么个数为dp[i - 1][j - 1]。即不需要考虑当前s子串和t子串的最后一位字母，所以只需要 dp[i-1][j-1]。
# 一部分是不用s[i - 1]来匹配，个数为dp[i - 1][j]。
# 这里可能有录友不明白了，为什么还要考虑 不用s[i - 1]来匹配，都相同了指定要匹配啊。
# 例如： s：bagg 和 t：bag ，s[3] 和 t[2]是相同的，但是字符串s也可以不用s[3]来匹配，即用s[0]s[1]s[2]组成的bag。
# 当然也可以用s[3]来匹配，即：s[0]s[1]s[3]组成的bag。
# 所以当s[i - 1] 与 t[j - 1]相等时，dp[i][j] = dp[i - 1][j - 1] + dp[i - 1][j];
# 当s[i - 1] 与 t[j - 1]不相等时，dp[i][j]只有一部分组成，不用s[i - 1]来匹配（就是模拟在s中删除这个元素），即：dp[i - 1][j]
# 所以递推公式为：dp[i][j] = dp[i - 1][j];
# 这里可能有录友还疑惑，为什么只考虑 “不用s[i - 1]来匹配” 这种情况， 不考虑 “不用t[j - 1]来匹配” 的情况呢。
# 这里大家要明确，我们求的是 s 中有多少个 t，而不是 求t中有多少个s，所以只考虑 s中删除元素的情况，即 不用s[i - 1]来匹配 的情况。
    
# 每次当初始化的时候，都要回顾一下dp[i][j]的定义，不要凭感觉初始化。
# dp[i][0]表示什么呢？
# dp[i][0] 表示：以i-1为结尾的s可以随便删除元素，出现空字符串的个数。
# 那么dp[i][0]一定都是1，因为也就是把以i-1为结尾的s，删除所有元素，出现空字符串的个数就是1。
# 再来看dp[0][j]，dp[0][j]：空字符串s可以随便删除元素，出现以j-1为结尾的字符串t的个数。
# 那么dp[0][j]一定都是0，s如论如何也变成不了t。
# 最后就要看一个特殊位置了，即：dp[0][0] 应该是多少。
# dp[0][0]应该是1，空字符串s，可以删除0个元素，变成空字符串t。
class Solution:
    def numDistinct(self, s: str, t: str) -> int:
        # dp[i][j]对应的是i-1和j-1，所以要到i+1，j+1
        dp = [[0] * (len(t)+1) for _ in range(len(s)+1)]
        for i in range(len(s)):
            dp[i][0] = 1
        for j in range(1, len(t)):
            dp[0][j] = 0
        for i in range(1, len(s)+1):
            for j in range(1, len(t)+1):
                if s[i-1] == t[j-1]:
                    # 当s[i - 1] 与 t[j - 1]相等时,dp[i][j]可以有两部分组成。
                    # 一部分是用s[i - 1]来匹配,那么个数为dp[i - 1][j - 1]。
                    # 即不需要考虑当前s子串和t子串的最后一位字母,所以只需要 dp[i-1][j-1]。
                    # 一部分是不用s[i - 1]来匹配,个数为dp[i - 1][j]。
                    dp[i][j] = dp[i-1][j-1] + dp[i-1][j]
                else:
                    # 当s[i - 1] 与 t[j - 1]不相等时,dp[i][j]只有一部分组成,
                    # 不用s[i - 1]来匹配(就是模拟在s中删除这个元素),即:dp[i - 1][j]
                    dp[i][j] = dp[i-1][j]
        return dp[-1][-1]

class SolutionDP2:
    """
    既然dp[i]只用到dp[i - 1]的状态,
    我们可以通过缓存dp[i - 1]的状态来对dp进行压缩,
    减少空间复杂度。
    (原理等同同于滚动数组)
    """
    
    def numDistinct(self, s: str, t: str) -> int:
        n1, n2 = len(s), len(t)
        if n1 < n2:
            return 0

        dp = [0 for _ in range(n2 + 1)]
        dp[0] = 1

        for i in range(1, n1 + 1):
            # 必须深拷贝
            # 不然prev[i]和dp[i]是同一个地址的引用
            prev = dp.copy()
            # 剪枝,保证s的长度大于等于t
            # 因为对于任意i,i > n1, dp[i] = 0
            # 没必要跟新状态。 
            end = i if i < n2 else n2
            for j in range(1, end + 1):
                if s[i - 1] == t[j - 1]:
                    dp[j] = prev[j - 1] + prev[j]
                else:
                    dp[j] = prev[j]
        return dp[-1]

In [ ]:
#39 *** (Medium) 72.编辑距离 Edit Distance
    # 给你两个单词 word1 和 word2，请你计算出将 word1 转换成 word2 所使用的最少操作数 。
    # 你可以对一个单词进行如下三种操作：
    # 插入一个字符
    # 删除一个字符
    # 替换一个字符
    # 示例 1：
    # 输入：word1 = "horse", word2 = "ros"
    # 输出：3
    # 解释： horse -> rorse (将 'h' 替换为 'r') rorse -> rose (删除 'r') rose -> ros (删除 'e')
    # 示例 2：
    # 输入：word1 = "intention", word2 = "execution"
    # 输出：5
    # 解释： intention -> inention (删除 't') inention -> enention (将 'i' 替换为 'e') enention -> exention (将 'n' 替换为 'x') exention -> exection (将 'n' 替换为 'c') exection -> execution (插入 'u')
# dp[i][j] 表示以下标i-1为结尾的字符串word1，和以下标j-1为结尾的字符串word2，最近编辑距离为dp[i][j]。
"""
    if (word1[i - 1] == word2[j - 1])
        不操作
    if (word1[i - 1] != word2[j - 1])
        增
        删
        换
"""
# if (word1[i - 1] == word2[j - 1]) 那么说明不用任何编辑，dp[i][j] 就应该是 dp[i - 1][j - 1]，即dp[i][j] = dp[i - 1][j - 1];
# if (word1[i - 1] != word2[j - 1])，此时就需要编辑了，如何编辑呢？
# 操作一：word1删除一个元素，那么就是以下标i - 2为结尾的word1 与 j-1为结尾的word2的最近编辑距离 再加上一个操作。
# 即 dp[i][j] = dp[i - 1][j] + 1;
# 操作二：word2删除一个元素，那么就是以下标i - 1为结尾的word1 与 j-2为结尾的word2的最近编辑距离 再加上一个操作。
# 即 dp[i][j] = dp[i][j - 1] + 1;
# 这里有同学发现了，怎么都是删除元素，添加元素去哪了。
# 操作三：替换元素，word1替换word1[i - 1]，使其与word2[j - 1]相同，此时不用增删加元素。
# 可以回顾一下，if (word1[i - 1] == word2[j - 1])的时候我们的操作 是 dp[i][j] = dp[i - 1][j - 1] 对吧。
# 那么只需要一次替换的操作，就可以让 word1[i - 1] 和 word2[j - 1] 相同。
# 所以 dp[i][j] = dp[i - 1][j - 1] + 1;
# 综上，当 if (word1[i - 1] != word2[j - 1]) 时取最小的，即：dp[i][j] = min({dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]}) + 1;
class Solution:
    def minDistance(self, word1: str, word2: str) -> int:
        # dp[i][j]对应的是i-1和j-1，所以要到i+1，j+1
        dp = [[0] * (len(word2)+1) for _ in range(len(word1)+1)]
        for i in range(len(word1)+1):
            dp[i][0] = i
        for j in range(len(word2)+1):
            dp[0][j] = j
        for i in range(1, len(word1)+1):
            for j in range(1, len(word2)+1):
                if word1[i-1] == word2[j-1]:
                    dp[i][j] = dp[i-1][j-1]
                else:
                    # if (word1[i - 1] != word2[j - 1])
                    # 操作一:word1删除一个元素 即 dp[i][j] = dp[i - 1][j] + 1
                    # 操作二:word2删除一个元素 即 dp[i][j] = dp[i][j - 1] + 1
                    # 操作三:替换元素,word1替换word1[i - 1],使其与word2[j - 1]相同
                    # 此时不用增删加元素。即 dp[i][j] = dp[i - 1][j - 1] + 1
                    # 综合三种操作,取最小就是
                    dp[i][j] = min(dp[i-1][j-1], dp[i-1][j], dp[i][j-1]) + 1
        return dp[-1][-1]

In [ ]:
#X42 *** (Medium) 221.Largest Square in a Matrix
    # Determine the area of the largest square of 1's in a binary matrix.
# dp[i][j] = 1 + min(dp[i - 1][j], dp[i - 1][j - 1], dp[i][j - 1])
from typing import List

def largest_square_in_a_matrix(matrix: List[List[int]]) -> int:
    if not matrix:
        return 0
    m, n = len(matrix), len(matrix[0])
    dp = [[0] * n for _ in range(m)]
    max_len = 0
    # Base case: If a cell in row 0 is 1, the largest square ending there has a
    # length of 1.
    for j in range(n):
        if matrix[0][j] == 1:
            dp[0][j] = 1
            max_len = 1
    # Base case: If a cell in column 0 is 1, the largest square ending there has
    # a length of 1.
    for i in range(m):
        if matrix[i][0] == 1:
            dp[i][0] = 1
            max_len = 1
    # Populate the rest of the DP table.
    for i in range(1, m):
        for j in range(1, n):
            if matrix[i][j] == 1:
                # The length of the largest square ending here is determined by 
                # the smallest square ending at the neighboring cells (left, 
                # top-left, top), plus 1 to include this cell.
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i - 1][j - 1], dp[i][j - 1])
            max_len = max(max_len, dp[i][j])
    return max_len ** 2

def largest_square_in_a_matrix_optimized(matrix: List[List[int]]) -> int:
    if not matrix:
        return 0
    m, n = len(matrix), len(matrix[0])
    prev_row = [0] * n
    max_len = 0
    # Iterate through the matrix.
    for i in range(m):
        curr_row = [0] * n
        for j in range(n):
            # Base cases: if we’re in row 0 or column 0, the largest square ending
            # here has a length of 1. This can be set by using the value in the
            # input matrix.
            if i == 0 or j == 0:
                curr_row[j] = matrix[i][j]
            else:
                if matrix[i][j] == 1:
                      # curr_row[j] = 1 + min(left, top-left, top)
                    curr_row[j] = 1 + min(curr_row[j - 1], prev_row[j - 1], prev_row[j])
            max_len = max(max_len, curr_row[j])
        # Update 'prev_row' with 'curr_row' values for the next iteration.
        prev_row, curr_row = curr_row, [0] * n
    return max_len ** 2

In [ ]:
#X40 *** (Medium) 647.回文子串 (连续) Palindromic Substrings
# 一个字符串
    # 给定一个字符串，你的任务是计算这个字符串中有多少个回文子串。
    # 具有不同开始位置或结束位置的子串，即使是由相同的字符组成，也会被视作不同的子串。
    # 示例 1：
    # 输入："abc"
    # 输出：3
    # 解释：三个回文子串: "a", "b", "c"
    # 示例 2：
    # 输入："aaa"
    # 输出：6
    # 解释：6个回文子串: "a", "a", "a", "aa", "aa", "aaa"
# 回文子串是要连续的,回文子序列可不是连续的!
# 647.回文子串 Palindromic Substrings
# 5.最长回文子串 Longest Palindromic Substring
# 布尔类型的dp[i][j]：表示区间范围[i,j] （注意是左闭右闭）的子串是否是回文子串，如果是dp[i][j]为true，否则为false。
# 在确定递推公式时，就要分析如下几种情况。
# 整体上是两种，就是s[i]与s[j]相等，s[i]与s[j]不相等这两种。
# 当s[i]与s[j]不相等，那没啥好说的了，dp[i][j]一定是false。
# 当s[i]与s[j]相等时，这就复杂一些了，有如下三种情况
# 情况一：下标i 与 j相同，同一个字符例如a，当然是回文子串
# 情况二：下标i 与 j相差为1，例如aa，也是回文子串
# 情况三：下标：i 与 j相差大于1的时候，例如cabac，此时s[i]与s[j]已经相同了，
# 我们看i到j区间是不是回文子串就看aba是不是回文就可以了，那么aba的区间就是 i+1 与 j-1区间，
# 这个区间是不是回文就看dp[i + 1][j - 1]是否为true。

# 遍历顺序可有有点讲究了。
# 首先从递推公式中可以看出，情况三是根据dp[i + 1][j - 1]是否为true，在对dp[i][j]进行赋值true的。
# dp[i + 1][j - 1] 在 dp[i][j]的左下角
# 如果这矩阵是从上到下，从左到右遍历，那么会用到没有计算过的dp[i + 1][j - 1]，
# 也就是根据不确定是不是回文的区间[i+1,j-1]，来判断了[i,j]是不是回文，那结果一定是不对的。
# 所以一定要从下到上，从左到右遍历，这样保证dp[i + 1][j - 1]都是经过计算的。
# 有的代码实现是优先遍历列，然后遍历行，其实也是一个道理，都是为了保证dp[i + 1][j - 1]都是经过计算的。
# 动态规划
class Solution:
    def countSubstrings(self, s: str) -> int:
        dp = [[False] * len(s) for _ in range(len(s))]
        result = 0
        for i in range(len(s)-1, -1, -1): #注意遍历顺序
            for j in range(i, len(s)):
                if s[i] == s[j]:
                    if j - i <= 1: #情况一 和 情况二
                        result += 1
                        dp[i][j] = True
                    elif dp[i+1][j-1]: #情况三
                        result += 1
                        dp[i][j] = True
        return result
# 动态规划:简洁版
# class Solution:
#     def countSubstrings(self, s: str) -> int:
#         dp = [[False] * len(s) for _ in range(len(s))]
#         result = 0
#         for i in range(len(s)-1, -1, -1): #注意遍历顺序
#             for j in range(i, len(s)):
#                 if s[i] == s[j] and (j - i <= 1 or dp[i+1][j-1]): 
#                     result += 1
#                     dp[i][j] = True
#         return result
# 双指针法
# 动态规划的空间复杂度是偏高的，我们再看一下双指针法。
# 首先确定回文串，就是找中心然后向两边扩散看是不是对称的就可以了。
# 在遍历中心点的时候，要注意中心点有两种情况。
# 一个元素可以作为中心点，两个元素也可以作为中心点。
# 那么有人同学问了，三个元素还可以做中心点呢。其实三个元素就可以由一个元素左右添加元素得到，四个元素则可以由两个元素左右添加元素得到。
# 所以我们在计算的时候，要注意一个元素为中心点和两个元素为中心点的情况。
# 这两种情况可以放在一起计算，但分别计算思路更清晰，我倾向于分别计算，代码如下
class Solution:
    def countSubstrings(self, s: str) -> int:
        result = 0
        for i in range(len(s)):
            result += self.extend(s, i, i, len(s)) #以i为中心
            result += self.extend(s, i, i+1, len(s)) #以i和i+1为中心
        return result
    
    def extend(self, s, i, j, n):
        res = 0
        while i >= 0 and j < n and s[i] == s[j]:
            i -= 1
            j += 1
            res += 1
        return res

# Longest Palindrome in a String
    # Return the longest palindromic substring within a given string.
    # Example:
    # Input: s = 'abccbaba'
    # Output: 'abccba'
def longest_palindrome_in_a_string(s: str) -> str:
    n = len(s)
    if n == 0:
        return ""
    dp = [[False] * n for _ in range(n)]
    max_len = 1
    start_index = 0
    # Base case: a single character is always a palindrome.
    for i in range(n):
        dp[i][i] = True
    # Base case: a substring of length two is a palindrome if both  
    # characters are the same.
    for i in range(n - 1):
        if s[i] == s[i + 1]:
            dp[i][i + 1] = True
            max_len = 2
            start_index = i
    # Find palindromic substrings of length 3 or greater.
    for substring_len in range(3, n + 1):
        # Iterate through each substring of length 'substring_len'.
        for i in range(n - substring_len + 1):
            j = i + substring_len - 1
            # If the first and last characters are the same, and the 
            # inner substring is a palindrome, then the current 
            # substring is a palindrome.
            if s[i] == s[j] and dp[i + 1][j - 1]:
                dp[i][j] = True
                max_len = substring_len
                start_index = i
    return s[start_index : start_index + max_len]

def longest_palindrome_in_a_string_expanding(s: str) -> str:
    n = len(s)
    start, max_len = 0, 0
    for center in range(n):
        # Check for odd-length palindromes.
        odd_start, odd_length = expand_palindrome(center, center, s)
        if odd_length > max_len:
            start = odd_start
            max_len = odd_length
        # Check for even-length palindromes.
        if center < n - 1 and s[center] == s[center + 1]:
            even_start, even_length = expand_palindrome(center, center + 1, s)
            if even_length > max_len:
                start = even_start
                max_len = even_length
    return s[start : start + max_len]

# Expands outward from the center of a base case to identify the start 
# index and length of the longest palindrome that extends from this 
# base case.
def expand_palindrome(left: int, right: int, s: str) -> Tuple[int, int]:
    while left > 0 and right < len(s) - 1 and s[left - 1] == s[right + 1]:
        left -= 1
        right += 1
    return left, right - left + 1

In [ ]:
#20 ??? (Medium) 139.单词拆分
    # 给定一个非空字符串 s 和一个包含非空单词的列表 wordDict，判定 s 是否可以被空格拆分为一个或多个在字典中出现的单词。
    # 说明：
    # 拆分时可以重复使用字典中的单词。
    # 你可以假设字典中没有重复的单词。
    # 示例 1：
    # 输入: s = "leetcode", wordDict = ["leet", "code"]
    # 输出: true
    # 解释: 返回 true 因为 "leetcode" 可以被拆分成 "leet code"。
    # 示例 2：
    # 输入: s = "applepenapple", wordDict = ["apple", "pen"]
    # 输出: true
    # 解释: 返回 true 因为 "applepenapple" 可以被拆分成 "apple pen apple"。
    # 注意你可以重复使用字典中的单词。
    # 示例 3：
    # 输入: s = "catsandog", wordDict = ["cats", "dog", "sand", "and", "cat"]
    # 输出: false
# dp[j] = dp[j] or (dp[j - len(word)] and word == s[j - len(word):j])
# dp[i] : 字符串长度为i的话，dp[i]为true，表示可以拆分为一个或多个在字典中出现的单词。
# 如果确定dp[j] 是true，且 [j, i] 这个区间的子串出现在字典里，那么dp[i]一定是true。（j < i ）。
# 所以递推公式是 if([j, i] 这个区间的子串出现在字典里 && dp[j]是true) 那么 dp[i] = true。
# 本题其实我们求的是排列数，为什么呢。 拿 s = "applepenapple", wordDict = ["apple", "pen"] 举例。
# "apple", "pen" 是物品，那么我们要求 物品的组合一定是 "apple" + "pen" + "apple" 才能组成 "applepenapple"。
# "apple" + "apple" + "pen" 或者 "pen" + "apple" + "apple" 是不可以的，那么我们就是强调物品之间顺序。
# 所以说，本题一定是 先遍历 背包，再遍历物品。

# 求组合数:动态规划:518.零钱兑换II 
# 求排列数:动态规划:377. 组合总和 Ⅳ 、动态规划:70. 爬楼梯进阶版(完全背包) 
# 求最小数:动态规划:322. 零钱兑换 、动态规划:279.完全平方数
# 回溯
# class Solution:
#     def backtracking(self, s: str, wordSet: set[str], startIndex: int) -> bool:
#         # 边界情况:已经遍历到字符串末尾,返回True
#         if startIndex >= len(s):
#             return True

#         # 遍历所有可能的拆分位置
#         for i in range(startIndex, len(s)):
#             word = s[startIndex:i + 1]  # 截取子串
#             if word in wordSet and self.backtracking(s, wordSet, i + 1):
#                 # 如果截取的子串在字典中,并且后续部分也可以被拆分成单词,返回True
#                 return True

#         # 无法进行有效拆分,返回False
#         return False

#     def wordBreak(self, s: str, wordDict: List[str]) -> bool:
#         wordSet = set(wordDict)  # 转换为哈希集合,提高查找效率
#         return self.backtracking(s, wordSet, 0)
# DP(版本一)
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        wordSet = set(wordDict)
        n = len(s)
        dp = [False] * (n + 1)  # dp[i] 表示字符串的前 i 个字符是否可以被拆分成单词
        dp[0] = True  # 初始状态,空字符串可以被拆分成单词

        for i in range(1, n + 1): # 遍历背包
            for j in range(i): # 遍历单词
                if dp[j] and s[j:i] in wordSet:
                    dp[i] = True  # 如果 s[0:j] 可以被拆分成单词,并且 s[j:i] 在单词集合中存在,则 s[0:i] 可以被拆分成单词
                    break
        return dp[n]
# DP(版本二)
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        dp = [False]*(len(s) + 1)
        dp[0] = True
        # 遍历背包
        for j in range(1, len(s) + 1):
            # 遍历单词
            for word in wordDict:
                if j >= len(word):
                    dp[j] = dp[j] or (dp[j - len(word)] and word == s[j - len(word):j])
        return dp[len(s)]

# dp[j] or ( ... ) (update rule):
# If either:
# dp[j] was already True (due to a previous word in wordDict), or
# The current word forms a valid segmentation (dp[j - len(word)] is True and the substring matches word),
# then dp[j] becomes True.